# Check Fitting Function

This notebook demonstrates the fitting of a single SDSS target with the analysis pipeline and checks that the pipeline results agree with results computed directly from SNCosmo. It is important to note that the full analysis uses nested sampling to determine model specific fitting priors for each individual object. We do not consider that process here, but restrict our scope to ensuring the core fitting functionality of the analysis pipeline returns consistent results.

In [ ]:
import sys

import numpy as np
import sncosmo
from SNData.sdss import sako18
from astropy.table import Table
from matplotlib import pyplot as plt

sys.path.insert(0, '../')
from analysis_pipeline import fit_lc

# Make sure SDSS data is locally available and registered with SNCosmo
sako18.download_module_data()
sako18.register_filters()


## Notes on data handling

For completeness, we note that the `SNData` package uses SDSS bandpasses from Doi et al. 2010. These are the same filters when fitting SDSS light curves.

In [ ]:
fig, axes = plt.subplots(1, 6, figsize=(24, 4))
for i, axis in enumerate(axes):
    for band in 'ugriz':
        band = sncosmo.get_bandpass('sdss_sako18_{}{}'.format(band, i + 1))
        axis.plot(band.wave, band.trans)
        axis.set_title('CCD Column {}'.format(i + 1))


We also note that the published SDSS data set includes flags notating outlier data points. These observations are automatically removed by `SNData`

## Picking an example target

For our demonstration, we specifically focus on targets that are spectroscopically confirmed SNe Ia. These can be isolated from the published SDSS data by selecting targets classified as `SNIa`.

In [ ]:
master_table = sako18.load_table('master')
master_table_indices = master_table['Classification'] == 'SNIa'
spec_confirmed_sn = master_table[master_table_indices]

print('A sample of the SDSS published results for '
      'spectroscopically confirmed SNe Ia:')
spec_confirmed_sn[:5]


Next, we pick fit an arbitrary object ID and retrieve the published values. 

In [ ]:
test_id = '739'
published_values = spec_confirmed_sn[spec_confirmed_sn['CID'] == test_id][0]
print('Example_id:', test_id)

print('\nPublished values:')
print('  z: ', published_values['zCMB'])
print('  t0:', published_values['MJDatPeakrmag'])
print('  x0:', published_values['x0SALT2zspec'])
print('  x1:', published_values['x1SALT2zspec'])
print('  c: ', published_values['cSALT2zspec'])

chisq_norm = published_values['chi2SALT2zspec'] / published_values['ndofSALT2zspec']
print('  Normalized chi-squared:', chisq_norm)

print('\nThe lightcurve:')
test_data = sako18.get_data_for_id(test_id)
fig, axes = plt.subplots(1, 5, figsize=(15, 3))
for i in range(5):
    band_data = test_data[test_data['FILT'] == i]
    axes[i].scatter(band_data['MJD'], band_data['MAG'])
    axes[i].errorbar(band_data['MJD'], band_data['MAG'], yerr=band_data['MERR'], linestyle='')
    axes[i].set_xlabel('MJD')

axes[0].set_ylabel('Mag')
plt.show()

print('\nA subset of the data table:')
test_data[:10]


# Light curve fitting

Using the data shown above, we fit an arbitrary light curve and check that the results we receive from the pipeline are the same as those we would receive from running SNCosmo directly.

In [ ]:
# Get an input table formatted for use with SNCosmo
input_table = sako18.get_sncosmo_input(test_id)

# Create a Salt2.0 model
source = sncosmo.get_source('salt2', version='2.0')
model = sncosmo.Model(source=source)
model.set(z=input_table.meta['redshift'])

# Run the fit with the pipeline
pipeline_results = fit_lc(
    input_table,
    model,
    ['t0', 'x0', 'x1', 'c'],
    modelcov=True)

print('\nPipeline results:')
print('  z: ', pipeline_results[2])
print('  t0:', pipeline_results[3])
print('  x0:', pipeline_results[4])
print('  x1:', pipeline_results[5])
print('  c: ', pipeline_results[6])
print('  Normalized chi-squared:', pipeline_results[10] / pipeline_results[11])

# Run the fit with SNCosmo
sncosmo_result, fitted_model = sncosmo.fit_lc(
    input_table,
    model, ['t0', 'x0', 'x1', 'c'],
    modelcov=True
)

print('\nSNCosmo values:')
print('  z: ', sncosmo_result.parameters[0])
print('  t0:', sncosmo_result.parameters[1])
print('  x0:', sncosmo_result.parameters[2])
print('  x1:', sncosmo_result.parameters[3])
print('  c: ', sncosmo_result.parameters[4])
print('  Normalized chi-squared:',
      sncosmo_result.chisq / sncosmo_result.ndof)

print('\nSNCosmo Plot:')
sncosmo.plot_lc(input_table, model=fitted_model, errors=sncosmo_result.errors)
plt.show()


Finally we take a look at the published values.

In [ ]:
print('\nPipeline results:')
print('  z: ', pipeline_results[2])
print('  t0:', pipeline_results[3])
print('  x0:', pipeline_results[4])
print('  x1:', pipeline_results[5])
print('  c: ', pipeline_results[6])
print('  Normalized chi-squared:', pipeline_results[10] / pipeline_results[11])

print('\nPublished values:')
print('  z: ', published_values['zCMB'])
print('  t0:', published_values['MJDatPeakrmag'])
print('  x0:', published_values['x0SALT2zspec'])
print('  x1:', published_values['x1SALT2zspec'])
print('  c: ', published_values['cSALT2zspec'])
print('  Normalized chi-squared:', chisq_norm)